Import libraries

In [0]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tqdm.notebook import tqdm
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image

Load images

In [0]:
images_directory = '/content/drive/My Drive/Water Bodies Satellite Images/Images'

In [0]:
images_to_load = []
for root, directory, file in os.walk(images_directory):
  for f in file:
    images_to_load.append(root + '/' + f)

In [0]:
images = []
for file in tqdm(sorted(images_to_load)):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(file)
    image = tf.keras.preprocessing.image.img_to_array(image)/255.0
    images.append(image)

images = np.array(images)

water_body_0.jpg
water_body_1.jpg
water_body_2.jpg
water_body_3.jpg
water_body_4.jpg



Load masks

In [0]:
masks_directory = '/content/drive/My Drive/Water Bodies Satellite Images/Masks'

In [0]:
masks_to_load = []
for root, directory, file in os.walk(masks_directory):
  for f in file:
    masks_to_load.append(root + '/' + f)

In [0]:
masks = []
for file in tqdm(sorted(masks_to_load)):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    mask = tf.keras.preprocessing.image.load_img(file)
    mask = tf.keras.preprocessing.image.img_to_array(image)/255.0
    masks.append(mask)

masks = np.array(masks)

water_body_0.jpg
water_body_1.jpg
water_body_2.jpg
water_body_3.jpg
water_body_4.jpg



U-NET

In [0]:
image_width = 256
image_height = 256
image_channels = 3
filter_multiplier = 8

In [0]:
inputs = Input(shape=(image_width, image_height, image_channels))
#1
first_convolution = Conv2D(filters=1*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(inputs)
first_convolution = Conv2D(filters=1*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(first_convolution)
first_pooling = MaxPooling2D(pool_size=(2,2))(first_convolution)
first_pooling = Dropout(0.5)(first_pooling)

#2
second_convolution = Conv2D(filters=2*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(first_pooling)
second_convolution = Conv2D(filters=2*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(second_convolution)
second_pooling = MaxPooling2D(pool_size=(2,2))(second_convolution)
second_pooling = Dropout(0.5)(second_pooling)

#3
third_convolution = Conv2D(filters=4*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(second_pooling)
third_convolution = Conv2D(filters=4*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(third_convolution)
third_pooling = MaxPooling2D(pool_size=(2,2))(third_convolution)
third_pooling = Dropout(0.5)(third_pooling)

#4
fourth_convolution = Conv2D(filters=8*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(third_pooling)
fourth_convolution = Conv2D(filters=8*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(fourth_convolution)
fourth_pooling = MaxPooling2D(pool_size=(2,2))(fourth_convolution)
fourth_pooling = Dropout(0.5)(fourth_pooling)

#5
fifth_convolution = Conv2D(filters=16*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(fourth_pooling)
fifth_convolution = Conv2D(filters=16*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(fifth_convolution)

#6
sixth_deconvolution = Conv2DTranspose(filters=8*filter_multiplier, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu' )(fifth_convolution)
sixth_convolution = concatenate([sixth_deconvolution, fourth_convolution])
sixth_convolution = Dropout(0.5)(sixth_convolution)
sixth_convolution = Conv2D(filters=8*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(sixth_convolution)
sixth_convolution = Conv2D(filters=8*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(sixth_convolution)


#7
seventh_deconvolution = Conv2DTranspose(filters=4*filter_multiplier, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu' )(sixth_convolution)
seventh_convolution = concatenate([third_convolution, seventh_deconvolution])
seventh_convolution = Dropout(0.5)(seventh_convolution)
seventh_convolution = Conv2D(filters=4*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(seventh_convolution)
seventh_convolution = Conv2D(filters=4*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(seventh_convolution)

#8
eigth_deconvolution = Conv2DTranspose(filters=2*filter_multiplier, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu' )(seventh_convolution)
eigth_convolution = concatenate([eigth_deconvolution, second_convolution])
eigth_convolution = Dropout(0.5)(eigth_convolution)
eigth_convolution = Conv2D(filters=2*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(eigth_convolution)
eigth_convolution = Conv2D(filters=2*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(eigth_convolution)

#9
ninth_deconvolution = Conv2DTranspose(filters=1*filter_multiplier, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu' )(eigth_convolution)
ninth_convolution = concatenate([ninth_deconvolution, first_convolution])
ninth_convolution = Dropout(0.5)(ninth_convolution)
ninth_convolution = Conv2D(filters=1*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(ninth_convolution)
ninth_convolution = Conv2D(filters=1*filter_multiplier, kernel_size=(3,3), padding='same', activation='relu' )(ninth_convolution)

#10
output = Conv2D(filters = 1, kernel_size=(1,1), padding='valid', activation='sigmoid')(ninth_convolution)

In [0]:
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='binary_crossentropy', metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

In [0]:
model.summary()

In [0]:
model.fit(images, masks, epochs = 5)

In [0]:
predictions = model.predict(images)

In [0]:
predictions.shape

In [0]:
predictions = predictions.reshape(predictions.shape[:-1])

In [0]:
predictions.shape

In [0]:
predictions[predictions>=predictions.mean()] = 1
predictions[predictions<predictions.mean()] = 0

In [0]:
plt.imshow(predictions[0], cmap='gray')

In [0]:
masks = masks.reshape(masks.shape[:-1])

In [0]:
plt.imshow(masks[0], cmap='gray', interpolation='nearest')

In [0]:
plt.imshow(images[0])

Calculate Volume

In [0]:
white = len(predictions[0][predictions[0]>=.5])
black = len(predictions[0][predictions[0]<.5])

In [0]:
water_percentage = white / (white+black)

In [0]:
water_percentage

In [0]:
real_area = 1000 #change this based on resolution of satellite data
water_area = real_area - water_percentage

In [0]:
average_depth = 62 #change this based on water body (m)